# Commuting patterns

In this worked example we demonstrate the use of FlowKit to investigate commuting patterns. We will use `meaningful_locations_aggregate` queries to calculate subscribers' home and work locations, following methods developed by [Isaacman et al.](https://doi.org/10.1007/978-3-642-21726-5_9) and [Zagatti et al.](https://doi.org/10.1016/j.deveng.2018.03.002).

### Load FlowClient and connect to FlowAPI

We start by importing FlowClient. We also import [folium](https://python-visualization.github.io/folium/), which we will use later to to visualise the data.

In [ ]:
import flowclient, folium

We must next generate an API access token using [FlowAuth](../../index.html#flowauth), and paste the token here as `TOKEN`. Once we have a token, we can start a connection to the FlowAPI system.

In [ ]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

### Create meaningful locations queries

We assign a day-of-week score of +1 to events which occur on weekdays (Monday-Friday), and a score of -1 to weekends (Saturday, Sunday). We assign an hour-of-day score of +1 to "working hours", which we define here as 08:00-17:00, and a score of -1 to evening

In [ ]:
tower_day_of_week_scores = {
    "monday": 1,
    "tuesday": 1,
    "wednesday": 1,
    "thursday": 1,
    "friday": 1,
    "saturday": -1,
    "sunday": -1,
}

tower_hour_of_day_scores=[-1, -1, -1, -1, -1, -1, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, -1, -1, -1, -1, -1]

meaningful_locations_labels={
    "evening": {
        "type": "Polygon",
        "coordinates": [
            [[-1, 1], [-1, -1], [-1e-06, -1], [-1e-06, 1]]
        ],
    },
    "daytime": {
        "type": "Polygon",
        "coordinates": [
            [[0, 1], [0, -1], [1, -1], [1, 1]]
        ],
    },
}

In [ ]:
evening_locations_spec = flowclient.meaningful_locations_aggregate(
    "2016-01-01",
    "2016-01-07",
    "evening",
    labels=meaningful_locations_labels,
    tower_day_of_week_scores=tower_day_of_week_scores,
    tower_hour_of_day_scores=tower_hour_of_day_scores,
    aggregation_unit="admin3"
)
daytime_locations_spec = flowclient.meaningful_locations_aggregate(
    "2016-01-01",
    "2016-01-07",
    "daytime",
    labels=meaningful_locations_labels,
    tower_day_of_week_scores=tower_day_of_week_scores,
    tower_hour_of_day_scores=tower_hour_of_day_scores,
    aggregation_unit="admin3"
)

In [ ]:
home_locations = flowclient.get_result(conn, evening_locations_spec)
work_locations = flowclient.get_result(conn, daytime_locations_spec)

In [ ]:
regions = flowclient.get_geography(conn, "admin3")

In [ ]:
m = folium.Map(location=[27.96834547, 85.960067737], zoom_start=7, tiles='cartodbpositron')

folium.Choropleth(
    geo_data=regions,
    name='Home locations',
    data=home_locations,
    columns=['pcod', 'total'],
    key_on='feature.properties.admin3pcod',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    legend_name='Home locations',
    highlight=True,
).add_to(m)

folium.Choropleth(
    geo_data=regions,
    name='Work locations',
    data=work_locations,
    columns=['pcod', 'total'],
    key_on='feature.properties.admin3pcod',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    legend_name='Work locations',
    highlight=True,
    show=False,
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
od_matrix_spec = flowclient.meaningful_locations_between_label_od_matrix(
    "2016-01-01",
    "2016-01-07",
    "evening",
    "day",
    labels=meaningful_locations_labels,
    tower_day_of_week_scores=tower_day_of_week_scores,
    tower_hour_of_day_scores=tower_hour_of_day_scores,
    aggregation_unit="admin3"
)

In [ ]:
od_matrix = flowclient.get_result(conn, od_matrix_spec)

In [ ]:
commuters_within_region = od_matrix[
    meaningful_locations_between_evening_day.pcod_from==meaningful_locations_between_evening_day.pcod_to
].drop(columns=["label_from", "label_to", "pcod_from"]).rename(columns={"pcod_to": "pcod"}).set_index("pcod")

commuters_into_region = (od_matrix.groupby("pcod_to").sum() - commuters_within_region).dropna().reset_index()
commuters_out_from_region = (od_matrix.groupby("pcod_from").sum() - commuters_within_region).dropna().reset_index()

In [ ]:
m = folium.Map(location=[27.96834547, 85.960067737], zoom_start=7, tiles='cartodbpositron')

folium.Choropleth(
    geo_data=regions,
    name='inflow',
    data=commuters_into_region,
    columns=['pcod_to', 'total'],
    key_on='feature.properties.admin3pcod',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    legend_name='inflow',
    highlight=True,
).add_to(m)

folium.Choropleth(
    geo_data=regions,
    name='outflow',
    data=commuters_out_from_region,
    columns=['pcod_from', 'total'],
    key_on='feature.properties.admin3pcod',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    legend_name='outflow',
    highlight=True,
    show=False,
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
meaningful_locations_between_evening_day[meaningful_locations_between_evening_day.pcod_from != meaningful_locations_between_evening_day.pcod_to]